<a href="https://colab.research.google.com/github/DmitryKutsev/cross-document_coreference/blob/main/classifiers/new_decision_tree.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [340]:
from sklearn import tree
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline


In [341]:
from IPython.display import SVG, display, HTML

In [342]:
from google.colab import drive
drive.mount('./content', force_remount=False)

Drive already mounted at ./content; to attempt to forcibly remount, call drive.mount("./content", force_remount=True).


In [343]:
!ls
import os
for i in os.listdir('content/'):
  print(i)

content			coref_df_anns60k.csv.3	new_anns60k.csv.3
coref_df_anns60k.csv	new_anns60k.csv		new_df_anns60k.csv
coref_df_anns60k.csv.1	new_anns60k.csv.1	sample_data
coref_df_anns60k.csv.2	new_anns60k.csv.2
.file-revisions-by-id
MyDrive
.shortcut-targets-by-id
.Trash


In [344]:
!wget https://raw.githubusercontent.com/DmitryKutsev/cross-document_coreference/main/data/coref_df_anns60k.csv
!wget https://raw.githubusercontent.com/DmitryKutsev/cross-document_coreference/main/data/new_anns60k.csv

--2021-05-16 12:43:30--  https://raw.githubusercontent.com/DmitryKutsev/cross-document_coreference/main/data/coref_df_anns60k.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1162863 (1.1M) [text/plain]
Saving to: ‘coref_df_anns60k.csv.4’

coref_df_anns60k.cs 100%[===================>]   1.11M  --.-KB/s    in 0.02s   

2021-05-16 12:43:30 (46.3 MB/s) - ‘coref_df_anns60k.csv.4’ saved [1162863/1162863]

--2021-05-16 12:43:30--  https://raw.githubusercontent.com/DmitryKutsev/cross-document_coreference/main/data/new_anns60k.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.110.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP reque

In [345]:
import csv
import re
from google.colab import files
from sklearn.model_selection import train_test_split
from sklearn.metrics.pairwise import cosine_similarity

In [346]:
with open('coref_df_anns60k.csv') as csvfile:
  reader = csv.reader(csvfile, delimiter='\t')
  with open('new_df_anns60k.csv', 'w') as write_file:
    writer = csv.writer(write_file, delimiter='\t')
    c = 0
    for row in reader:
      new_row = [i if i and not re.match(r'\t+?', i) else '-' for i in row]
      # print(new_row)
      if len(new_row) > 9:
        # print(new_row[:9])
        new_row = new_row[:9]
      writer.writerow(new_row)


In [347]:
files.download('new_df_anns60k.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [348]:
# anns_df = pd.read_csv('coref_df_anns60k.csv', delimiter='\t', error_bad_lines=False, header=None)
anns_df = pd.read_csv('new_df_anns60k.csv', delimiter='\t')

In [349]:
anns_df

,TOKEN,TOPIC_NUM,POST_NUM,SENT_NUM,TOKEN_NUM,IS_ANSWER,HEAD,NER,COREFERENCE_CLUSTER
0,обсуждаем,1,0,0,0,0,обсуждаем,0,0.0
1,препарат,1,0,0,1,0,обсуждаем,0,0.0
2,рисперидон,1,0,0,2,0,препарат,1,2.0
3,рисполепт,1,0,0,3,0,рисперидон,1,3.0
4,сперидан,1,0,0,4,0,рисполепт,1,4.0
...,...,...,...,...,...,...,...,...,...
26492,и,22,68,0,2385,0,пошлины,0,0.0
26493,таможенные,22,68,0,2386,0,пошлины,0,0.0
26494,пошлины,22,68,0,2387,0,есть,0,0.0
26495,как,22,68,0,2388,0,есть,0,0.0


In [350]:
import numpy as np

In [351]:
cor_df = pd.read_csv('content/MyDrive/coref_fasttext60k.csv', delimiter=' ')

In [352]:

np.matrix(cor_df.HEAD_VECT[0]).A[0]

array([-2.08233938e-01,  9.96180058e-01, -2.79894114e-01, -9.09143448e-01,
        3.49961907e-01,  1.22820950e+00, -4.53999013e-01, -4.96644318e-01,
       -5.06227851e-01, -3.50297660e-01, -2.68426925e-01, -3.18647206e-01,
       -4.94759202e-01, -4.05789405e-01, -6.19022734e-02,  5.76916397e-01,
        6.45665407e-01, -6.29692733e-01, -2.28346493e-02,  4.74443585e-01,
        2.75724858e-01,  1.00444555e-01, -2.81267107e-01,  7.38310993e-01,
        1.64172634e-01, -1.40955552e-01, -1.34702528e+00, -2.28699774e-01,
       -2.00072408e-01,  1.53492257e-01,  2.15266779e-01, -3.75066400e-01,
        9.79104459e-01,  1.09640598e-01,  4.11819309e-01, -5.48374474e-01,
        5.74254513e-01,  1.78177617e-02,  2.20505193e-01,  8.06000233e-01,
       -3.45855951e-01, -9.48401261e-03, -1.23907340e+00, -7.95203969e-02,
       -7.97536790e-01, -5.43104485e-02,  4.87657279e-01,  5.39513767e-01,
       -2.17315450e-01, -3.24471802e-01, -3.16938460e-02, -4.50928695e-02,
       -3.87996256e-01,  

In [353]:
cosine_similarity([np.matrix(cor_df.TOKEN_VECT[2]).A[0]], [np.matrix(cor_df.TOKEN_VECT[3]).A[0]])[-1][-1]

0.8436395154681279

# **VECTORS HANDLING**

In [354]:
cor_df.TOKEN_VECT = [np.matrix(i).A[0] for i in cor_df.TOKEN_VECT]

In [355]:
cor_df.HEAD_VECT = [np.matrix(i).A[0] for i in cor_df.HEAD_VECT]

In [356]:
cor_df.SENT_VECT = [np.matrix(i).A[0] for i in cor_df.SENT_VECT]

In [357]:
cor_df.HEAD_CHILDS_VECT = [np.matrix(i).A[0] for i in cor_df.HEAD_CHILDS_VECT]

In [358]:
cor_df['NER'] = anns_df['NER']
cor_df['COREFERENCE_CLUSTER'] = anns_df['COREFERENCE_CLUSTER']
cor_df

,TOKEN,TOKEN_VECT,IS_ANSWER,TOPIC_NUM,POST_NUM,SENT_NUM,TOKEN_NUM,ANIMACY,CASE,GENDER,PERSON,POS,DEPENDENCY,HEAD,HEAD_ANIMACY,HEAD_CASE,HEAD_GENDER,HEAD_POS,HEAD_VECT,SENT_VECT,HEAD_CHILDS_VECT,NER,COREFERENCE_CLUSTER
0,обсуждаем,"[-0.208233938, 0.996180058, -0.279894114, -0.9...",0,1,0,0,0,NaN,NaN,NaN,1per,VERB,ROOT,обсуждаем,NaN,NaN,NaN,VERB,"[-0.208233938, 0.996180058, -0.279894114, -0.9...","[0.01863711, 0.2306802, 0.14600733, -0.0191193...","[-0.17725945, 0.94834274, 0.90867615, 0.850609...",0,0.0
1,препарат,"[-0.17725945, 0.94834274, 0.90867615, 0.850609...",0,1,0,0,1,inan,nomn,masc,NaN,NOUN,obj,обсуждаем,NaN,NaN,NaN,VERB,"[-0.208233938, 0.996180058, -0.279894114, -0.9...","[0.01863711, 0.2306802, 0.14600733, -0.0191193...","[-0.17725945, 0.94834274, 0.90867615, 0.850609...",0,0.0
2,рисперидон,"[0.15259643, 1.0673474, 0.84192747, -0.0775155...",0,1,0,0,2,anim,nomn,masc,NaN,NOUN,appos,препарат,inan,nomn,masc,NOUN,"[-0.17725945, 0.94834274, 0.90867615, 0.850609...","[0.01863711, 0.2306802, 0.14600733, -0.0191193...","[0.15259643, 1.0673474, 0.84192747, -0.0775155...",1,2.0
3,рисполепт,"[0.012426247, 1.0408522, 0.6913349, -0.1638528...",0,1,0,0,3,inan,gent,femn,NaN,NOUN,parataxis,рисперидон,anim,nomn,masc,NOUN,"[0.15259643, 1.0673474, 0.84192747, -0.0775155...","[0.01863711, 0.2306802, 0.14600733, -0.0191193...","[0.012426247, 1.0408522, 0.6913349, -0.1638528...",1,3.0
4,сперидан,"[0.08634768, 0.893356, 0.55127096, -0.06507572...",0,1,0,0,4,NaN,NaN,masc,NaN,PRTS,conj,рисполепт,inan,gent,femn,NOUN,"[0.012426247, 1.0408522, 0.6913349, -0.1638528...","[0.01863711, 0.2306802, 0.14600733, -0.0191193...","[0.02878256, 0.29778534, 0.18375699, -0.021691...",1,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26492,и,"[0.5048941, 0.81449467, 0.29763442, 0.60393727...",0,22,68,0,7,NaN,NaN,NaN,NaN,CONJ,cc,пошлины,inan,nomn,femn,NOUN,"[0.7365944, -0.48505062, -0.00257558, 0.168963...","[0.100772209, 0.0629534647, -0.035381455, -0.0...","[0.41243815, 0.190811, -0.11933009, 0.04242903...",0,0.0
26493,таможенные,"[0.73242044, -0.24206167, -0.6556247, -0.47665...",0,22,68,0,8,NaN,nomn,NaN,NaN,ADJF,amod,пошлины,inan,nomn,femn,NOUN,"[0.7365944, -0.48505062, -0.00257558, 0.168963...","[0.100772209, 0.0629534647, -0.035381455, -0.0...","[0.41243815, 0.190811, -0.11933009, 0.04242903...",0,0.0
26494,пошлины,"[0.7365944, -0.48505062, -0.00257558, 0.168963...",0,22,68,0,9,inan,nomn,femn,NaN,NOUN,nsubj,есть,NaN,NaN,NaN,INFN,"[-0.0050342, 0.14796825, 0.08416066, -0.776544...","[0.100772209, 0.0629534647, -0.035381455, -0.0...","[0.57842094, 0.09196201, -0.19555457, -0.26701...",0,0.0
26495,как,"[1.0349948, 0.73793733, -0.38374668, -0.544819...",0,22,68,0,10,NaN,NaN,NaN,NaN,CONJ,advmod,есть,NaN,NaN,NaN,INFN,"[-0.0050342, 0.14796825, 0.08416066, -0.776544...","[0.100772209, 0.0629534647, -0.035381455, -0.0...","[0.57842094, 0.09196201, -0.19555457, -0.26701...",0,0.0


In [359]:
df = cor_df.iloc[:6541,:]

In [106]:
len(df.loc[df['NER'] == 1])

283

In [108]:
len(df.loc[df['NER'] == 3])

54

In [109]:
len(df.loc[df['NER'].isin([1,2,3])])

426

In [107]:
len(df.loc[df['NER'] == 2])

89

In [110]:
df.loc[df['NER'].isin([1,2,3])]

,TOKEN,TOKEN_VECT,IS_ANSWER,TOPIC_NUM,POST_NUM,SENT_NUM,TOKEN_NUM,ANIMACY,CASE,GENDER,PERSON,POS,DEPENDENCY,HEAD,HEAD_ANIMACY,HEAD_CASE,HEAD_GENDER,HEAD_POS,HEAD_VECT,SENT_VECT,HEAD_CHILDS_VECT,NER,COREFERENCE_CLUSTER
2,рисперидон,"[0.15259643, 1.0673474, 0.84192747, -0.0775155...",0,1,0,0,2,anim,nomn,masc,NaN,NOUN,appos,препарат,inan,nomn,masc,NOUN,"[-0.17725945, 0.94834274, 0.90867615, 0.850609...","[0.01863711, 0.2306802, 0.14600733, -0.0191193...","[0.15259643, 1.0673474, 0.84192747, -0.0775155...",1,2.0
3,рисполепт,"[0.15259643, 1.0673474, 0.84192747, -0.0775155...",0,1,0,0,3,inan,gent,femn,NaN,NOUN,parataxis,рисперидон,anim,nomn,masc,NOUN,"[-0.17725945, 0.94834274, 0.90867615, 0.850609...","[0.01863711, 0.2306802, 0.14600733, -0.0191193...","[0.15259643, 1.0673474, 0.84192747, -0.0775155...",1,3.0
4,сперидан,"[0.15259643, 1.0673474, 0.84192747, -0.0775155...",0,1,0,0,4,NaN,NaN,masc,NaN,PRTS,conj,рисполепт,inan,gent,femn,NOUN,"[-0.17725945, 0.94834274, 0.90867615, 0.850609...","[0.01863711, 0.2306802, 0.14600733, -0.0191193...","[0.15259643, 1.0673474, 0.84192747, -0.0775155...",1,4.0
142,рисполепта,"[0.15259643, 1.0673474, 0.84192747, -0.0775155...",0,1,1,13,7,inan,nomn,femn,NaN,NOUN,nmod,дозировку,inan,accs,femn,NOUN,"[-0.17725945, 0.94834274, 0.90867615, 0.850609...","[0.01863711, 0.2306802, 0.14600733, -0.0191193...","[0.15259643, 1.0673474, 0.84192747, -0.0775155...",1,142.0
344,рисполепта,"[0.15259643, 1.0673474, 0.84192747, -0.0775155...",1,1,2,4,64,inan,nomn,femn,NaN,NOUN,nmod,приеме,inan,loct,masc,NOUN,"[-0.17725945, 0.94834274, 0.90867615, 0.850609...","[0.01863711, 0.2306802, 0.14600733, -0.0191193...","[0.15259643, 1.0673474, 0.84192747, -0.0775155...",1,344.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6502,он,"[0.15259643, 1.0673474, 0.84192747, -0.0775155...",1,5,14,0,300,NaN,nomn,masc,3per,NPRO,nsubj,убрал,NaN,NaN,masc,VERB,"[-0.17725945, 0.94834274, 0.90867615, 0.850609...","[0.01863711, 0.2306802, 0.14600733, -0.0191193...","[0.15259643, 1.0673474, 0.84192747, -0.0775155...",3,270.0
6517,тиапридом,"[0.15259643, 1.0673474, 0.84192747, -0.0775155...",1,5,15,0,315,anim,ablt,masc,NaN,NOUN,nmod,отравлении,inan,loct,neut,NOUN,"[-0.17725945, 0.94834274, 0.90867615, 0.850609...","[0.01863711, 0.2306802, 0.14600733, -0.0191193...","[0.15259643, 1.0673474, 0.84192747, -0.0775155...",1,319.0
6526,препарат,"[0.15259643, 1.0673474, 0.84192747, -0.0775155...",1,5,16,0,324,inan,nomn,masc,NaN,NOUN,obj,отнесли,NaN,NaN,NaN,VERB,"[-0.17725945, 0.94834274, 0.90867615, 0.850609...","[0.01863711, 0.2306802, 0.14600733, -0.0191193...","[0.15259643, 1.0673474, 0.84192747, -0.0775155...",2,319.0
6535,метоклопрамида,"[0.15259643, 1.0673474, 0.84192747, -0.0775155...",1,5,16,1,333,inan,gent,masc,NaN,NOUN,nmod,близнец,anim,nomn,masc,NOUN,"[-0.17725945, 0.94834274, 0.90867615, 0.850609...","[0.01863711, 0.2306802, 0.14600733, -0.0191193...","[0.15259643, 1.0673474, 0.84192747, -0.0775155...",1,337.0


# **SEMANTIC AXIS**

In [360]:
df['TOPIC_NUM'] == 0

0       False
1       False
2       False
3       False
4       False
        ...  
6536    False
6537    False
6538    False
6539    False
6540    False
Name: TOPIC_NUM, Length: 6541, dtype: bool

In [361]:
(df['TOPIC_NUM'] == 0)
axis_df = df.query('POST_NUM == 0 and TOKEN_NUM == 2 and SENT_NUM == 0')
axis_df

,TOKEN,TOKEN_VECT,IS_ANSWER,TOPIC_NUM,POST_NUM,SENT_NUM,TOKEN_NUM,ANIMACY,CASE,GENDER,PERSON,POS,DEPENDENCY,HEAD,HEAD_ANIMACY,HEAD_CASE,HEAD_GENDER,HEAD_POS,HEAD_VECT,SENT_VECT,HEAD_CHILDS_VECT,NER,COREFERENCE_CLUSTER
2,рисперидон,"[0.15259643, 1.0673474, 0.84192747, -0.0775155...",0,1,0,0,2,anim,nomn,masc,NaN,NOUN,appos,препарат,inan,nomn,masc,NOUN,"[-0.17725945, 0.94834274, 0.90867615, 0.850609...","[0.01863711, 0.2306802, 0.14600733, -0.0191193...","[0.15259643, 1.0673474, 0.84192747, -0.0775155...",1,2.0
576,зелдокс,"[0.2465947, 1.0097265, 0.31869438, 0.07363212,...",0,2,0,0,2,inan,nomn,masc,NaN,NOUN,appos,препарат,inan,nomn,masc,NOUN,"[-0.17725945, 0.94834274, 0.90867615, 0.850609...","[0.03948287, 0.24082112, 0.22588146, -0.017884...","[0.2465947, 1.0097265, 0.31869438, 0.07363212,...",1,2.0
1870,оланзапин,"[-0.082683861, 1.2017089, 0.67815673, -0.22599...",0,3,0,0,2,inan,gent,femn,NaN,NOUN,appos,препарат,inan,nomn,masc,NOUN,"[-0.17725945, 0.94834274, 0.90867615, 0.850609...","[0.01915067, 0.0875298, 0.09080387, 0.00378973...","[-0.082683861, 1.2017089, 0.67815673, -0.22599...",1,2.0
5555,галоперидол,"[-0.03019532, 1.2394575, 0.18222564, -0.215572...",0,4,0,0,2,inan,nomn,masc,NaN,NOUN,obj,обсуждаем,NaN,NaN,NaN,VERB,"[-0.208233938, 0.996180058, -0.279894114, -0.9...","[-0.03747297, 0.5554195, 0.15133946, -0.011017...","[-0.10372739, 1.0939001, 0.5454509, 0.3175185,...",1,2.0
6200,тиаприд,"[0.27581334, 0.80696917, 1.0271364, -0.1037090...",0,5,0,0,2,anim,nomn,masc,NaN,NOUN,appos,препарат,inan,nomn,masc,NOUN,"[-0.17725945, 0.94834274, 0.90867615, 0.850609...","[0.10855111, 0.28903672, 0.2550475, -0.0276597...","[0.27581334, 0.80696917, 1.0271364, -0.1037090...",1,3.0


In [362]:
sum(df.query('POST_NUM == 0 and TOKEN_NUM == 2 and SENT_NUM == 0').TOKEN_VECT)/len(df.query('POST_NUM == 0 and TOKEN_NUM == 2 and SENT_NUM == 0').TOKEN_VECT)

array([ 0.11242506,  1.06504189,  0.60962812, -0.1098321 , -0.25309914,
        0.18837714, -0.29014948, -0.66894132,  0.12588727, -0.36141735,
       -0.05396164, -0.35577641, -0.14144993, -0.15987718, -0.68407827,
       -0.04969584,  0.04704007,  0.06857683,  0.14719775, -0.42784514,
        0.12565804, -0.09923978, -0.46527826,  0.02228641, -0.29075186,
        0.19580094, -0.26369618, -0.26473205, -0.52919418,  0.07394732,
       -0.1234136 , -0.1277977 , -0.02898717,  0.14748353,  0.18349706,
       -0.02373637,  0.25928119, -0.35122524, -0.0759414 ,  0.94526116,
        0.08296286,  0.64028589, -0.28902657,  0.08378753, -0.44904899,
       -0.24238156,  0.36918995, -0.20184582, -0.20304089, -0.099586  ,
        0.13669602,  0.38995985, -0.19354978, -0.37518906,  0.09664967,
        0.01878619, -0.22422224, -0.06534558,  0.09065832, -0.05079969,
       -0.09178696, -0.39066393, -0.03679946, -0.37383265, -0.24993176,
       -0.28516305,  0.65741815, -0.18487955,  0.46176331,  0.12

In [363]:
token_axe = sum(axis_df.TOKEN_VECT)/len(axis_df.TOKEN_VECT)
head_axe = sum(axis_df.HEAD_VECT)/len(axis_df.HEAD_VECT)
head_childs_axe = sum(axis_df.HEAD_CHILDS_VECT)/len(axis_df.HEAD_CHILDS_VECT)
sent_axe = sum(axis_df.SENT_VECT)/len(axis_df.SENT_VECT)

In [365]:
df.TOKEN_VECT[710]

array([nan])

In [366]:
def make_similarity(df, col, axe):
  new_col = []
  for ind, vect in enumerate(df[col]):
    try:
      new_col.append(cosine_similarity([axe], [vect])[-1][-1])
    except ValueError:
      new_col.append(0)
  df[col] = new_col



In [367]:
make_similarity(df, 'TOKEN_VECT', token_axe)
make_similarity(df, 'HEAD_VECT', head_axe)
make_similarity(df, 'HEAD_CHILDS_VECT', head_childs_axe)
make_similarity(df, 'SENT_VECT', sent_axe)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/s

In [370]:
df

,TOKEN,TOKEN_VECT,IS_ANSWER,TOPIC_NUM,POST_NUM,SENT_NUM,TOKEN_NUM,ANIMACY,CASE,GENDER,PERSON,POS,DEPENDENCY,HEAD,HEAD_ANIMACY,HEAD_CASE,HEAD_GENDER,HEAD_POS,HEAD_VECT,SENT_VECT,HEAD_CHILDS_VECT,NER,COREFERENCE_CLUSTER
0,обсуждаем,0.424228,0,1,0,0,0,NaN,NaN,NaN,1per,VERB,ROOT,обсуждаем,NaN,NaN,NaN,VERB,0.481209,0.904469,0.564925,0,0.0
1,препарат,0.403455,0,1,0,0,1,inan,nomn,masc,NaN,NOUN,obj,обсуждаем,NaN,NaN,NaN,VERB,0.481209,0.904469,0.564925,0,0.0
2,рисперидон,0.933164,0,1,0,0,2,anim,nomn,masc,NaN,NOUN,appos,препарат,inan,nomn,masc,NOUN,0.987311,0.904469,0.917969,1,2.0
3,рисполепт,0.863874,0,1,0,0,3,inan,gent,femn,NaN,NOUN,parataxis,рисперидон,anim,nomn,masc,NOUN,0.415625,0.904469,0.856327,1,3.0
4,сперидан,0.806514,0,1,0,0,4,NaN,NaN,masc,NaN,PRTS,conj,рисполепт,inan,gent,femn,NOUN,0.438658,0.904469,0.806874,1,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6536,и,0.195650,1,5,16,1,334,NaN,NaN,NaN,NaN,CONJ,cc,всё,NaN,NaN,NaN,PRCL,0.337437,0.358325,0.245561,0,0.0
6537,всё,0.250212,1,5,16,1,335,NaN,NaN,NaN,NaN,PRCL,conj,близнец,anim,nomn,masc,NOUN,0.420502,0.358325,0.499390,0,0.0
6538,а,0.267463,1,5,17,0,336,NaN,NaN,NaN,NaN,CONJ,cc,убирает,NaN,NaN,NaN,VERB,0.341443,0.466717,0.713536,0,0.0
6539,тиаприд,0.889868,1,5,17,0,337,anim,nomn,masc,NaN,NOUN,nsubj,убирает,NaN,NaN,NaN,VERB,0.341443,0.466717,0.713536,1,341.0


In [371]:
df.describe()

,TOKEN_VECT,IS_ANSWER,TOPIC_NUM,POST_NUM,SENT_NUM,TOKEN_NUM,HEAD_VECT,SENT_VECT,HEAD_CHILDS_VECT,NER,COREFERENCE_CLUSTER
count,6541.000000,6541.000000,6541.000000,6541.00000,6541.000000,6541.000000,6541.000000,6541.000000,6541.000000,6541.000000,6541.000000
mean,0.352978,0.929369,2.830148,15.74377,4.258829,1180.104877,0.366729,0.511356,0.453299,0.095245,66.903532
std,0.148008,0.256228,0.910212,12.06082,5.718161,1106.784078,0.103371,0.115372,0.145562,0.403665,370.716460
min,0.000000,0.000000,1.000000,0.00000,0.000000,0.000000,0.000000,0.257439,0.000000,0.000000,0.000000
25%,0.249610,1.000000,2.000000,6.00000,1.000000,234.000000,0.310923,0.443867,0.375299,0.000000,0.000000
50%,0.333842,1.000000,3.000000,14.00000,2.000000,800.000000,0.362414,0.497136,0.457233,0.000000,0.000000
75%,0.416451,1.000000,3.000000,23.00000,5.000000,2033.000000,0.418611,0.554191,0.538834,0.000000,0.000000
max,0.934259,1.000000,5.000000,49.00000,31.000000,3668.000000,0.987311,0.928780,0.918316,3.000000,3653.000000


In [375]:
for_factorize = 'ANIMACY	CASE	GENDER	PERSON	POS	DEPENDENCY	HEAD	HEAD_ANIMACY	HEAD_CASE	HEAD_GENDER	HEAD_POS'.split()
for_factorize

['ANIMACY',
 'CASE',
 'GENDER',
 'PERSON',
 'POS',
 'DEPENDENCY',
 'HEAD',
 'HEAD_ANIMACY',
 'HEAD_CASE',
 'HEAD_GENDER',
 'HEAD_POS']

In [385]:

_, lbl = pd.factorize(df.ANIMACY)
print(lbl[0])
_

inan


array([-1,  0,  1, ..., -1,  1, -1])

In [386]:
mapping_dict = {}
for col_name in for_factorize:
  col, lbl = pd.factorize(df[col_name])
  df[col_name] = col
  mapping_dict[col_name] = lbl
mapping_dict  

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


{'ANIMACY': Index(['inan', 'anim'], dtype='object'),
 'CASE': Index(['nomn', 'gent', 'datv', 'accs', 'loct', 'ablt', 'loc2'], dtype='object'),
 'DEPENDENCY': Index(['ROOT', 'obj', 'appos', 'parataxis', 'conj', 'advmod', 'ccomp', 'iobj',
        'csubj', 'case', 'obl', 'det', 'nsubj', 'cc', 'discourse', 'fixed',
        'mark', 'amod', 'advcl', 'xcomp', 'nummod', 'nmod', 'nsubj:pass',
        'nummod:gov', 'cop', 'acl:relcl', 'acl', 'aux', 'flat:foreign', 'punct',
        'obl:agent', 'flat', 'aux:pass', 'orphan', 'expl'],
       dtype='object'),
 'GENDER': Index(['masc', 'femn', 'neut'], dtype='object'),
 'HEAD': Index(['обсуждаем', 'препарат', 'рисперидон', 'рисполепт', 'знаю', 'стоило',
        'меня', 'нет', 'проблем', 'лекарством',
        ...
        'ухудшение', 'увеличением', 'дозы', 'рецидив', 'очередной', 'рацион',
        'продолжает', 'увеличивать', 'катадолоном', 'ответьте'],
       dtype='object', length=1312),
 'HEAD_ANIMACY': Index(['inan', 'anim'], dtype='object'),
 'HE

In [ ]:
# X = cor_df.drop(['COREFERENCE_CLUSTER', 'NER', 'TOKEN'], axis=1)
# X_test = cor_df_test.drop(['COREFERENCE_CLUSTER', 'NER', 'TOKEN'], axis=1)
# labels_dict = {}
# test_labels_dict = {}

# for i in X.columns:
#   X[i], lbl = pd.factorize(X[i])
#   labels_dict[i] = lbl

# for i in X_test.columns:
#   X_test[i], lbl = pd.factorize(X_test[i])
#   test_labels_dict[i] = lbl 

# y = cor_df['COREFERENCE_CLUSTER']
# y, y_label = pd.factorize(y)

# y_test = cor_df_test['COREFERENCE_CLUSTER']
# y_test, y_test_label = pd.factorize(y_test)

In [387]:
train, test = train_test_split(df, test_size=0.2)

In [ ]:
train = train.reset_index()

In [401]:

train = train.iloc[:,:24].drop(['TOKEN', 'level_0', 'index'], axis=1)

In [402]:
test = test.reset_index()

In [406]:
test

,index,TOKEN,TOKEN_VECT,IS_ANSWER,TOPIC_NUM,POST_NUM,SENT_NUM,TOKEN_NUM,ANIMACY,CASE,GENDER,PERSON,POS,DEPENDENCY,HEAD,HEAD_ANIMACY,HEAD_CASE,HEAD_GENDER,HEAD_POS,HEAD_VECT,SENT_VECT,HEAD_CHILDS_VECT,NER,COREFERENCE_CLUSTER
0,6016,я,0.299118,1,4,7,4,392,-1,0,-1,0,5,12,1213,-1,-1,0,0,0.356102,0.504414,0.512747,0,0.0
1,3761,ulegające,0.263834,1,3,23,4,1877,-1,-1,-1,-1,-1,28,760,-1,-1,-1,-1,0.393213,0.557775,0.398415,0,0.0
2,4538,вес,0.455346,1,3,29,1,2654,0,3,0,-1,1,1,660,-1,-1,-1,5,0.280684,0.496733,0.434450,0,0.0
3,3350,пила,0.564324,1,3,18,7,1466,0,0,1,-1,1,0,447,0,0,1,1,0.280499,0.492354,0.729530,0,0.0
4,2571,мг,0.611221,1,3,8,9,687,0,0,0,-1,1,1,418,1,1,0,1,0.403573,0.586127,0.627152,0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1304,4198,млн.евро,0.384900,1,3,23,22,2314,0,0,2,-1,1,21,907,0,3,1,1,0.375539,0.482218,0.409693,0,0.0
1305,270,писать,0.285828,0,1,1,27,7,-1,-1,-1,-1,6,18,123,-1,-1,1,0,0.388319,0.456387,0.428397,0,0.0
1306,3658,повышением,0.335698,1,3,22,1,1774,0,5,2,-1,1,10,731,-1,-1,-1,0,0.555029,0.534767,0.578581,0,0.0
1307,4869,теряешь,0.281536,1,3,36,3,2985,-1,-1,-1,2,0,6,686,-1,-1,-1,0,0.487177,0.558765,0.314980,0,0.0


In [412]:
test = test.iloc[:,:24].drop(['TOKEN', 'index'], axis=1)

In [411]:
test

,TOKEN_VECT,IS_ANSWER,TOPIC_NUM,POST_NUM,SENT_NUM,TOKEN_NUM,ANIMACY,CASE,GENDER,PERSON,POS,DEPENDENCY,HEAD,HEAD_ANIMACY,HEAD_CASE,HEAD_GENDER,HEAD_POS,HEAD_VECT,SENT_VECT,HEAD_CHILDS_VECT,NER,COREFERENCE_CLUSTER
0,0.299118,1,4,7,4,392,-1,0,-1,0,5,12,1213,-1,-1,0,0,0.356102,0.504414,0.512747,0,0.0
1,0.263834,1,3,23,4,1877,-1,-1,-1,-1,-1,28,760,-1,-1,-1,-1,0.393213,0.557775,0.398415,0,0.0
2,0.455346,1,3,29,1,2654,0,3,0,-1,1,1,660,-1,-1,-1,5,0.280684,0.496733,0.434450,0,0.0
3,0.564324,1,3,18,7,1466,0,0,1,-1,1,0,447,0,0,1,1,0.280499,0.492354,0.729530,0,0.0
4,0.611221,1,3,8,9,687,0,0,0,-1,1,1,418,1,1,0,1,0.403573,0.586127,0.627152,0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1304,0.384900,1,3,23,22,2314,0,0,2,-1,1,21,907,0,3,1,1,0.375539,0.482218,0.409693,0,0.0
1305,0.285828,0,1,1,27,7,-1,-1,-1,-1,6,18,123,-1,-1,1,0,0.388319,0.456387,0.428397,0,0.0
1306,0.335698,1,3,22,1,1774,0,5,2,-1,1,10,731,-1,-1,-1,0,0.555029,0.534767,0.578581,0,0.0
1307,0.281536,1,3,36,3,2985,-1,-1,-1,2,0,6,686,-1,-1,-1,0,0.487177,0.558765,0.314980,0,0.0


In [ ]:
X = 

In [ ]:
clf = tree.DecisionTreeClassifier()
clf.fit(X, y)

In [ ]:
clf_ent = tree.DecisionTreeClassifier(criterion='entropy')
clf_ent.fit(X, y)

In [ ]:
from graphviz import Source
from sklearn import tree

In [ ]:
# tree.plot_tree(clf)

In [ ]:
# graph = Source(tree.export_graphviz(clf, out_file=None, 
#                                     feature_names=list(X), class_names=uniques, filled=True))

# display(SVG(graph.pipe(format='svg')))

In [ ]:
clf.predict(X_test)

In [ ]:
y_test

In [ ]:
X_test